In [ ]:
import os
import pickle
import numpy as np
import cv2 as cv

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from metrics import compute_distance
from histograms import compute_descriptor 
import evaluation
import utils

%matplotlib inline

In [ ]:
# Week 3 Task 5 ~~ QST2
dataset_dir = "{}/BBDD".format(os.getcwd()) # dataset path
l_queryset_dir = "{}/qst2_week3/L".format(os.getcwd()) # l
r_queryset_dir = "{}/qst2_week3/R".format(os.getcwd()) # r
l_mask_dir = "{}/qst2_week3/L_masks".format(os.getcwd()) # l mask
r_mask_dir = "{}/qst2_week3/R_masks".format(os.getcwd()) # r mask
# testset_dir = "{}/qst1_w1".format(os.getcwd()) # test set path
database_dir = "{}/database".format(os.getcwd()) # database path
result_dir = "{}/qst2_week3/result".format(os.getcwd()) ## result path

metric = 'hellinger_kernel'
descriptor = 'block_histogram_4_rgb' 
# gt_file = 'gt_corresps_task5_week3.pkl'
k = 10

# read masks

l_masks = [os.path.join(l_mask_dir, f) for f in sorted(os.listdir(l_mask_dir)) if f.endswith(".png")]
r_masks = [os.path.join(r_mask_dir, f) for f in sorted(os.listdir(r_mask_dir)) if f.endswith(".png")]
l_queries = [os.path.join(l_queryset_dir, f) for f in sorted(os.listdir(l_queryset_dir)) if f.endswith(".jpg")]
r_queries = [os.path.join(r_queryset_dir, f) for f in sorted(os.listdir(r_queryset_dir)) if f.endswith(".jpg")]
m_queries = [0, 1, 3, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 18, 20, 21, 23, 24, 25, 28, 29] # remember to change this line for each dataset

results = []
histograms = utils.load_database(descriptor, dataset_dir, database_dir)
# ground_truth = utils.load_pickle(gt_file)

for idx, (lq, rq, lm, rm) in enumerate(zip(l_queries, r_queries, l_masks, r_masks)):
    print("Query {}: {}, ground-truth: {}".format(idx, lq[-9:-4], str(0)))    
    l_query_hist = compute_descriptor(descriptor, lq, lm)   
    if idx not in m_queries:
        r_query_hist = compute_descriptor(descriptor, rq, rm)
        
    l_k_nearest = utils.k_nearest_search(histograms, l_query_hist, 0, metric, k)
    if idx not in m_queries:
        r_k_nearest = utils.k_nearest_search(histograms, r_query_hist, 0, metric, k)
    
    result = []
    print("{}-most similar images:".format(k))
    l_result = []
    for i, image in enumerate(l_k_nearest):
        print("{}. {}.jpg, score = {}".format(i + 1, str(image[-1]).zfill(5), image[0]))
        l_result.append(image[-1])
    result.append(l_result)
    if idx not in m_queries:
        r_result = []
        for i, image in enumerate(r_k_nearest):
            print("{}. {}.jpg, score = {}".format(i + 1, str(image[-1]).zfill(5), image[0]))
            r_result.append(image[-1])
        result.append(r_result)
    results.append(result)
    print("==================================")
print(results)    
utils.save_pickle(result_dir + '/result.pkl', results)

In [ ]:
# Week 3 Task 4 ~~ QST1
dataset_dir = "{}/BBDD".format(os.getcwd()) # dataset path
queryset_dir = "{}/qst1_week3/denoised".format(os.getcwd()) # query set path
mask_dir = "{}/qst1_week3/masks".format(os.getcwd()) # masks
# testset_dir = "{}/qst1_w1".format(os.getcwd()) # test set path
database_dir = "{}/database".format(os.getcwd()) # database path
result_dir = "{}/qst1_week3/result".format(os.getcwd()) ## result path

metric = 'hellinger_kernel'
descriptor = 'block_histogram_4_rgb' 
# gt_file = 'gt_corresps_task4_week3.pkl'
k = 10

masks = [os.path.join(mask_dir, f) for f in sorted(os.listdir(mask_dir)) if f.endswith(".png")]
queries = [os.path.join(queryset_dir, f) for f in sorted(os.listdir(queryset_dir)) if f.endswith(".jpg")]

color_results = []
histograms = utils.load_database(descriptor, dataset_dir, database_dir)
# ground_truth = utils.load_pickle(gt_file) # for test set we dont have any ground truth
q = []
for idx, (q, m) in enumerate(zip(queries, masks)):
    print("Query {}: {}, ground-truth: {}".format(idx, q[-9:-4], str(0).zfill(5)))
    
    query_hist = compute_descriptor(descriptor, q, m)
    q = query_hist
    k_nearest = utils.k_nearest_search(histograms, query_hist, 0, metric, k)
    
    print("{}-most similar images:".format(k))
    result = []
    temp = []
    for i, image in enumerate(k_nearest):
        print("{}. {}.jpg, score = {}".format(i + 1, str(image[-1]).zfill(5), image[0]))
        result.append(image[-1])
        if i == k - 1:
            temp.append(result)
            color_results.append(temp)   
    print("==================================")
print(color_results)    
utils.save_pickle(result_dir + '/result.pkl', color_results)

In [ ]:
# Week 3 Task 2 ~~ text descriptor
dataset_dir = "{}/BBDD".format(os.getcwd()) # dataset path
# queryset_dir = "{}/qsd1_w2".format(os.getcwd()) # query set path
text_dir = "{}/qst1_week3/extracted_text".format(os.getcwd()) # masks
# testset_dir = "{}/qst1_w1".format(os.getcwd()) # test set path
database_dir = "{}/database".format(os.getcwd()) # database path
result_dir = "{}/qst1_week3/result".format(os.getcwd()) ## result path

metric = 'levenshtein'
descriptor = 'text' 
# gt_file = 'gt_corresps_task4_week3.pkl'
k = 10

texts = [os.path.join(text_dir, f) for f in sorted(os.listdir(text_dir)) if f.endswith(".txt")]

text_results = []
indexed_texts = utils.load_text(descriptor, dataset_dir, database_dir)
painters = [t[0] if len(t) > 0 else "" for t in indexed_texts]

# ground_truth = utils.load_pickle(gt_file) # for test set we dont have any ground truth

for idx, t in enumerate(texts):
    print("Query {}: {}, ground-truth: {}".format(idx, t[-9:-4], str(0).zfill(5)))
    
    painter = ""
    with open(t, 'r') as f:
        text = f.read().strip()
        print(text)
        painter = text
#         if len(text) != 0:
#             deli = "', \"" if text.endswith("\")") else "', '"
#             tup = tuple(str(x.strip()) for x in text[2:-2].split(deli))
#             painter_name = tup[0] + " " + tup[1]
      
    k_nearest = utils.k_nearest_search(painters, painter, 0, metric, k)
    
    print("{}-most similar images:".format(k))
    result = []
    temp = []
    for i, image in enumerate(k_nearest):
        print("{}. {}.jpg, score = {}".format(i + 1, str(image[-1]).zfill(5), image[0]))
        result.append(image[-1])
        if i == k - 1:
            temp.append(result)
            text_results.append(temp)   
    print("==================================")
print(text_results)    
utils.save_pickle(result_dir + '/result.pkl', text_results)